# Human Protein Atlas Image Classification


Image classification of microscope slides based on mixed protein patterns.

### Requirements

- keras
- tensorflow
- numpy
- pydot
- pandas
- OpenCV (opencv-python)

In [3]:
# libraries

from keras.layers import Dense, Activation
from keras.models import Sequential

import cv2              # openCV image processing
import numpy as np      # linear algebra
import pandas as pd     # data processing
import pydot            # graphing/visualization

import os
import gc
import random

Using TensorFlow backend.


### Labels

In [4]:
NUCLEOPLASM                   = 0
NUCLEAR_MEMBRANE              = 1
NUCLEOLI                      = 2
NUCLEOLI_FIBRILLAR_CENTER     = 3
NUCLEAR_SPECKLES              = 4
NUCLEAR_BODIES                = 5
ENDOPLASMIC_RETICULUM         = 6
GOLGI_APPARATUS               = 7
PEROXISOMES                   = 8
ENDOSOMES                     = 9
LYSOSOMES                     = 10
INTERMEDIATE_FILAMENTS        = 11
ACTIN_FILAMENTS               = 12
FOCAL_ADHESION_SITES          = 13
MICROTUBULES                  = 14
MICROTUBULE_ENDS              = 15
CYTOKINETIC_BRIDGE            = 16
MITOTIC_SPINDLE               = 17
MICROTUBULE_ORGANIZING_CENTER = 18
CENTROSOME                    = 19
LIPID_DROPLETS                = 20
PLASMA_MEMBRANE               = 21
CELL_JUNCTIONS                = 22
MITOCHONDRIA                  = 23
AGGRESOME                     = 24
CYTOSOL                       = 25
CYTOPLASMIC_BODIES            = 26
RODS_AND_RINGS                = 27

### Some Constants

In [15]:
LABELS_DIR = 'dataset'
TRAIN_DIR = 'dataset/train'
TEST_DIR = 'dataset/test'
TRAIN_SUBSET_SIZE = 10 # Selection of images to train from (chosen at random, max size is 31072)

### Image Preprocessing

The first step is to take our dataset and prepare it so that it's ready for our model.

Images are split into four filters/layers:
- **green**: the protein of interest
- **blue**: the nucleus
- **red**: the microtubules
- **yellow**: the endoplasmic reticulum

For this project we will mostly be interested in the **green** filter, which will be used to predict the label, while the other filters will be used as references.

In [23]:
# Get images by layer

# training set images
train_green  = [(TRAIN_DIR+'/{}').format(i) for i in os.listdir(TRAIN_DIR) if 'green'  in i]

# Select TRAIN_SUBSET_SIZE images from the training set at random
random.shuffle(train_green)
train_green = train_green[:TRAIN_SUBSET_SIZE]
train_green_ids = [i[:-10].replace((TRAIN_DIR+'/'),'') 
                       for i in train_green] # remove '_green.png' and '_green.tif'

# Retrieve the other three layers for our subset (NB: unsorted)
train_blue =   [(TRAIN_DIR+'/{}').format(i) for i in os.listdir(TRAIN_DIR)
                  if ((i[:-9] in train_green_ids) and ('blue' in i))]
train_red =    [(TRAIN_DIR+'/{}').format(i) for i in os.listdir(TRAIN_DIR)
                  if ((i[:-8] in train_green_ids) and ('red' in i))]
train_yellow = [(TRAIN_DIR+'/{}').format(i) for i in os.listdir(TRAIN_DIR)
                  if ((i[:-11] in train_green_ids) and ('yellow' in i))]

# test set images
test_green  = [(TEST_DIR+'/{}').format(i) for i in os.listdir(TEST_DIR) if 'green'  in i]
test_blue   = [(TEST_DIR+'/{}').format(i) for i in os.listdir(TEST_DIR) if 'blue'   in i]
test_red    = [(TEST_DIR+'/{}').format(i) for i in os.listdir(TEST_DIR) if 'red'    in i]
test_yellow = [(TEST_DIR+'/{}').format(i) for i in os.listdir(TEST_DIR) if 'yellow' in i]

In [28]:
# Testing the above by seeing if the IDs are the same

train_green.sort()
train_blue.sort()
train_red.sort()
train_yellow.sort()
print(train_green, end='\n\n')
print(train_blue, end='\n\n')
print(train_red, end='\n\n')
print(train_yellow, end='\n\n')

['dataset/train/2da04718-bba0-11e8-b2b9-ac1f6b6435d0_green.png', 'dataset/train/37e8ba80-bba4-11e8-b2b9-ac1f6b6435d0_green.png', 'dataset/train/57996924-bbc7-11e8-b2bc-ac1f6b6435d0_green.png', 'dataset/train/5917a040-bbae-11e8-b2ba-ac1f6b6435d0_green.png', 'dataset/train/67322146-bba9-11e8-b2ba-ac1f6b6435d0_green.png', 'dataset/train/8c85ea40-bbae-11e8-b2ba-ac1f6b6435d0_green.png', 'dataset/train/bde0f150-bbba-11e8-b2ba-ac1f6b6435d0_green.png', 'dataset/train/c4959e82-bbac-11e8-b2ba-ac1f6b6435d0_green.png', 'dataset/train/ccf6cf7e-bbac-11e8-b2ba-ac1f6b6435d0_green.png', 'dataset/train/f1455a1a-bbc2-11e8-b2bc-ac1f6b6435d0_green.png']

['dataset/train/2da04718-bba0-11e8-b2b9-ac1f6b6435d0_blue.png', 'dataset/train/37e8ba80-bba4-11e8-b2b9-ac1f6b6435d0_blue.png', 'dataset/train/57996924-bbc7-11e8-b2bc-ac1f6b6435d0_blue.png', 'dataset/train/5917a040-bbae-11e8-b2ba-ac1f6b6435d0_blue.png', 'dataset/train/67322146-bba9-11e8-b2ba-ac1f6b6435d0_blue.png', 'dataset/train/8c85ea40-bbae-11e8-b2ba-ac1